In [1]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from pycdhit import cd_hit, read_clstr
from remove_duplicates_from_negatives import (
    duplicates_analysis,
    duplicates_removal,
    balance_negatives,
    verify_results,
)

# Konfiguracja nazw plików
# POSITIVES_FILE = "positives_filtered_with_upstream.csv"
# NEGATIVES_FILE = "negatives_diverse_13k.csv"

POSITIVES_FILE = "./data/positives_with_upstream.csv"
NEGATIVES_FILE = "./data/negatives_diverse_100k.csv"

In [2]:
class EcolisORFDetector:
    """
    Zoptymalizowany detektor sORF dla E. coli.
    Zawiera poprawioną obsługę CD-HIT i upstream_30bp.
    """

    SHINE_DALGARNO = ["AGGAGG", "GGAGG", "GAGG", "GGAG", "GGA"]
    ECOLI_START_CODONS = {"ATG": 1.0, "GTG": 0.52, "TTG": 0.28, "CTG": 0.15}

    def __init__(self):
        self.model = RandomForestClassifier(
            n_estimators=100, max_depth=10, random_state=42
        )
        self.scaler = StandardScaler()

    def shine_dalgarno_score(self, upstream_seq):
        """Oblicza wynik dopasowania Shine-Dalgarno"""
        if pd.isna(upstream_seq) or len(str(upstream_seq)) < 5:
            return 0.0

        upstream_seq = str(upstream_seq).upper()
        rbs_score = 0
        u_len = len(upstream_seq)

        for rbs in self.SHINE_DALGARNO:
            pos = upstream_seq.rfind(rbs)
            if pos != -1:
                # Dystans od końca RBS do końca sekwencji upstream (czyli do startu sORF)
                distance = u_len - (pos + len(rbs))
                # Optymalny dystans u E. coli to ok. 5-12 pz
                if 5 <= distance <= 13:
                    score = (13 - distance) / 8.0  # Heurystyka punktacji
                    rbs_score = max(rbs_score, score)
        return rbs_score

    def extract_features(self, sorf_seq, upstream_seq):
        """Ekstrakcja cech numerycznych"""
        features = {}
        seq = str(sorf_seq).upper()
        u_seq = str(upstream_seq).upper() if pd.notna(upstream_seq) else ""

        # 1. RBS Score (Kluczowe dla E. coli)
        features["rbs_score"] = self.shine_dalgarno_score(u_seq)

        # 2. Start Codon Strength
        start_codon = seq[:3]
        features["start_strength"] = self.ECOLI_START_CODONS.get(start_codon, 0.0)

        # 3. Parametry podstawowe
        features["length_norm"] = min(len(seq) / 300.0, 1.0)
        features["gc_content"] = (
            (seq.count("G") + seq.count("C")) / len(seq) if len(seq) > 0 else 0
        )
        features["at_bias"] = (
            (seq.count("A") + seq.count("T")) / len(seq) if len(seq) > 0 else 0
        )

        # 4. Prosty indeks stabilności (np. GC upstream)
        features["upstream_gc"] = (
            (u_seq.count("G") + u_seq.count("C")) / len(u_seq) if len(u_seq) > 0 else 0
        )

        return list(features.values())

    def cluster_sequences(self, fasta_file, output_prefix, threshold=0.9):
        """
        Uruchamia CD-HIT i zwraca listę ID reprezentantów.
        Naprawiony błąd parsowania pliku .clstr
        """
        # Uruchomienie CD-HIT (zapisuje pliki na dysku)
        # c=threshold, G=1 (global), d=0 (pełny nagłówek)
        try:
            cd_hit(i=fasta_file, o=output_prefix, c=threshold, d=0, sc=1, G=1)
        except Exception as e:
            print(f"Ostrzeżenie CD-HIT (może być problem ze ścieżką): {e}")
            # Sprawdź czy plik wyjściowy powstał mimo błędu wrappera
            if not os.path.exists(output_prefix + ".clstr"):
                raise e

        # Ręczne parsowanie pliku .clstr (niezawodne)
        clstr_file = f"{output_prefix}.clstr"
        representatives = []

        with open(clstr_file, "r") as f:
            for line in f:
                # Szukamy linii z gwiazdką '*', która oznacza reprezentanta klastra
                # Format: 0	297nt, >seq_123... *
                if "*" in line:
                    # Wyciągamy ID sekwencji (pomiędzy > a ...)
                    match = re.search(r">(.+?)\.\.\.", line)
                    if match:
                        representatives.append(match.group(1))
                    else:
                        # Fallback dla krótkich nagłówków bez kropek
                        match_simple = re.search(r">(.+?)\s", line) or re.search(
                            r">(.+?)$", line
                        )
                        if match_simple:
                            # Usuwamy ewentualny znak * na końcu jeśli złapał się regexem
                            clean_id = match_simple.group(1).replace("*", "").strip()
                            representatives.append(clean_id)

        return representatives


def df_to_fasta(df, filename, seq_col="sorf"):
    """Pomocnicza funkcja zapisu do FASTA"""
    with open(filename, "w") as f:
        for idx, row in df.iterrows():
            # Używamy indeksu jako ID, żeby łatwo filtrować
            f.write(f">seq_{idx}\n{row[seq_col]}\n")

In [6]:
# 1. Wczytanie danych
print("Wczytywanie danych...")
pos_df = pd.read_csv(POSITIVES_FILE)
neg_df = pd.read_csv(NEGATIVES_FILE)

print(f"Liczba surowych pozytywów: {len(pos_df)}")
print(f"Liczba surowych negatywów: {len(neg_df)}")

# 2. Klastrowanie Pozytywów (Usuwanie duplikatów)
print("\nUruchamianie CD-HIT dla pozytywów...")
df_to_fasta(pos_df, "positives.fasta", seq_col="sorf")

detector = EcolisORFDetector()
# Próg 0.9 (90% identyczności) usuwa bardzo podobne sekwencje
keep_ids = detector.cluster_sequences(
    "positives.fasta", "positives_clustered", threshold=0.9
)

# Parsowanie ID z powrotem na indeksy (format: seq_123 -> 123)
keep_indices = [int(x.replace("seq_", "")) for x in keep_ids if "seq_" in x]

# Filtrowanie DataFrame
pos_filtered = pos_df.loc[keep_indices].copy()
print(f"Pozytywy po klastrowaniu (unikalne): {len(pos_filtered)}")

# 3. Balansowanie klas (Undersampling)
# Chcemy tyle samo negatywów, co unikalnych pozytywów
# Sprawdzenie obecności zmiennych
if "neg_df" not in locals():
    raise ValueError(
        "BŁĄD: Zmienna 'neg_df' nie istnieje! Wczytaj najpierw dane negatywne."
    )
if "pos_filtered" not in locals():
    print("⚠️ Ostrzeżenie: Brak 'pos_filtered'")
else:
    target_n = len(pos_filtered)

duplicates_analysis(neg_df)
neg_df_unique = duplicates_removal(neg_df)
neg_balanced = balance_negatives(neg_df_unique, target_n)


print(f"✓ Wczytano zmienną 'neg_df'. Rozmiar oryginalny: {len(neg_df):,} próbek")
print(f"✓ Cel balansowania (liczba unikalnych pozytywów): {target_n}")

print(f"Negatywy po balansowaniu: {len(neg_balanced)}")

# Sprawdzenie końcowe
print(f"\nFINAŁOWY ZBIÓR: {len(pos_filtered)} sORF vs {len(neg_balanced)} Non-sORF")

Wczytywanie danych...


/tmp/ipykernel_30350/1952969864.py:3: DtypeWarning: Columns (13,29) have mixed types. Specify dtype option on import or set low_memory=False.
  pos_df = pd.read_csv(POSITIVES_FILE)


Liczba surowych pozytywów: 100000
Liczba surowych negatywów: 98347

Uruchamianie CD-HIT dla pozytywów...
Pozytywy po klastrowaniu (unikalne): 3818
Liczba duplikatów (dokładnych powtórzeń sekwencji): 16,638
Procent duplikatów: 16.92%

Top 10 najczęściej powtarzanych sekwencji sORF:
   1. Sekwencja #ATGAATAAAC...TGA   pojawia się    44 razy
   2. Sekwencja #ATGCTGTATT...TAA   pojawia się    41 razy
   3. Sekwencja #ATGCTGAACT...TAG   pojawia się    38 razy
   4. Sekwencja #ATGACTCTAG...TAA   pojawia się    35 razy
   5. Sekwencja #ATGCTACGCG...TAA   pojawia się    35 razy
   6. Sekwencja #ATGCGAAAAC...TAG   pojawia się    35 razy
   7. Sekwencja #ATGTAACGAA...TAG   pojawia się    35 razy
   8. Sekwencja #ATGAGTGAAA...TAG   pojawia się    35 razy
   9. Sekwencja #ATGTTGAAGC...TGA   pojawia się    35 razy
   10. Sekwencja #ATGATTCGTC...TAG   pojawia się    35 razy

Statystyki długości sekwencji:
   Min: 30 bp, Max: 198 bp, Średnia: 90.4 bp
✓ Usunięto duplikaty. Nowy rozmiar zbioru: 81,709


In [4]:
# 4. Generowanie macierzy X, y
X = []
y = []

print("\nGenerowanie cech...")

# Przetwarzanie pozytywów
# Używamy kolumny 'upstream_30bp' zgodnie z Twoimi plikami
for _, row in pos_filtered.iterrows():
    feats = detector.extract_features(row["sorf"], row.get("upstream_30bp", ""))
    X.append(feats)
    y.append(1)

# Przetwarzanie negatywów
for _, row in neg_balanced.iterrows():
    feats = detector.extract_features(row["sorf"], row.get("upstream_30bp", ""))
    X.append(feats)
    y.append(0)

X = np.array(X)
y = np.array(y)

# 5. Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Rozmiar zbioru treningowego: {X_train.shape}")
print(f"Rozmiar zbioru testowego: {X_test.shape}")

# 6. Trening modelu
print("\nTrenowanie klasyfikatora Random Forest...")
detector.scaler.fit(X_train)
X_train_scaled = detector.scaler.transform(X_train)
detector.model.fit(X_train_scaled, y_train)

# 7. Ewaluacja
X_test_scaled = detector.scaler.transform(X_test)
y_pred = detector.model.predict(X_test_scaled)
y_probs = detector.model.predict_proba(X_test_scaled)[:, 1]

print("\n" + "=" * 50)
print("WYNIKI KLASYFIKACJI sORF (E. coli)")
print("=" * 50)
print(classification_report(y_test, y_pred, target_names=["Non-sORF", "sORF"]))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_probs):.4f}")

# Feature importance
feature_names = [
    "rbs_score",
    "start_strength",
    "length_norm",
    "gc_content",
    "at_bias",
    "upstream_gc",
]
importances = detector.model.feature_importances_
indices = np.argsort(importances)[::-1]

print("\nNajważniejsze cechy:")
for i in range(len(feature_names)):
    print(f"{i+1}. {feature_names[indices[i]]}: {importances[indices[i]]:.4f}")


Generowanie cech...
Rozmiar zbioru treningowego: (6108, 6)
Rozmiar zbioru testowego: (1528, 6)

Trenowanie klasyfikatora Random Forest...

WYNIKI KLASYFIKACJI sORF (E. coli)
              precision    recall  f1-score   support

    Non-sORF       0.95      0.96      0.95       764
        sORF       0.96      0.95      0.95       764

    accuracy                           0.95      1528
   macro avg       0.95      0.95      0.95      1528
weighted avg       0.95      0.95      0.95      1528

ROC AUC Score: 0.9903

Najważniejsze cechy:
1. length_norm: 0.4681
2. upstream_gc: 0.2853
3. start_strength: 0.1249
4. rbs_score: 0.0596
5. at_bias: 0.0333
6. gc_content: 0.0289
